In [1]:
import pandas as analytics
import numpy as maths
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.models import Sequential
from data_extraction import df_data , df_items
import warnings
warnings.filterwarnings("ignore")

2024-04-26 19:54:38.963930: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 19:54:38.964288: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 19:54:38.966581: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 19:54:38.997024: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 19:54:39.520310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
embedding_dimension = 20

input_dimension = 18
model = Sequential()
model.add(Embedding(input_dim = input_dimension, output_dim=embedding_dimension))
model.add(GRU(50, activation = 'softmax'))
model.add(Dense(input_dimension, activation='softmax'))
model.compile(loss = 'crossentropy',optimizer='adam', metrics=['accuracy'])


2024-04-26 19:54:40.567058: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
def find_interesting_set(df_user):
    df_user_summary = analytics.DataFrame(df_user[df_user.columns[5:]].value_counts(normalize = True)).reset_index()
    df_interests = df_user_summary.iloc[:int(len(df_user_summary)*0.30)].drop('proportion',axis = 1)
    df_interesting = analytics.DataFrame()
    for row in range(len(df_interests)):
        df_temp = df_items
        for col in df_interests.columns :
            df_temp = df_temp[df_temp[col] == df_interests[col].iloc[row]]
        df_interesting = analytics.concat([df_interesting,df_temp])
    df_interesting = df_interesting.drop_duplicates()
    return df_interesting

In [4]:
def calculate_rewards(rewards,f):
    if f > 0 : value = 1
    else : value = -.2
    
    
    rewards.append(gamma**count + value)
    return rewards

In [5]:
def update_grad(predictions,rewards,df_remaining,df_interesting):
    first_time = False
    
    try :
        predictions.shape[1]
        last_prediction = predictions[-1]
        
    except IndexError:
        first_time = True
        last_prediction = predictions
        
    updates = 0

    rewards = [0] + rewards
    
    for count in range(len(rewards)-1):
        if not first_time : log_vec = predictions[count-1]
        else : log_vec = predictions
        update = gamma ** (count) * rewards[count+1] * maths.log(log_vec)
        updates = updates + update
    
    last_prediction = last_prediction + eta/len(rewards) * updates

    probabilities = maths.exp(last_prediction) / maths.exp(last_prediction).sum()
    threshold = probabilities.argsort()[-2:]
    req = maths.zeros(len(probabilities))
    for x in threshold :
        req[x] = 1 
    df_temp = df_remaining[df_remaining.columns[2:]]
    df_temp = df_temp[df_temp[df_temp.columns[0]] == req[0]] 
    for k in range(len(df_temp.columns)):
        df_temp = df_temp[df_temp[df_temp.columns[k]] == req[k]]
        
    if len(df_temp):
        df_temp = df_temp.sample(n = 1)
        df_temp = df_remaining.iloc[df_temp.index]
    else :
        df_temp = df_interesting.sample(n = 1)
        print(".",end = "")
    
    return df_temp
    
    

In [6]:
all_genres = df_items.columns[2:]

In [7]:
users = df_data.groupby('user_id').agg({'movie_id':lambda x:x.count()}).sort_values('movie_id').iloc[:20].reset_index()['user_id'].to_list()

In [8]:
N_counts = {}
N_values = [10,15,20]
# N_values = [10]

reinforce = True
gamma = 0.9
eta = 0.8

for N in N_values :
    print("\nFor",N)
    hit_rates = []
    for user in users :
        print("\n\tUser #",users.index(user)+1, "User ID #",user)
        hit_rate = 0
        df_user = df_data[df_data['user_id'] == user].reset_index(drop = True).sort_values('timestamp')
        df_user_movies = df_items[df_items['movie_id'].isin(df_user['movie_id'])]
        df_user = df_user.merge(df_user_movies,on='movie_id')
        
        if reinforce : df_train = df_user_movies.sample(n = 1)
        else : df_train = df_user_movies.sample(frac = 0.8)

        
        print("\tCompiling the interesting set ...",end = " ")
        if reinforce : df_interesting = find_interesting_set(df_user)
        else : df_interesting = df_user_movies
        print("Compiled")

        df_remaining = df_items[~df_items['movie_id'].isin(df_train['movie_id'])]
        rewards = []
        predictions = maths.array(df_train[df_train.columns[2:]].mean())
        predictions = maths.exp(predictions)/maths.exp(predictions).sum()


        
        for count in range(len(df_user)):
            if count == 0 :
                last_prediction = predictions
            else :
                last_prediction = predictions[-1]
                
            selected_genres = all_genres[last_prediction.argsort()[-2:]]
            df_candidates = df_remaining[df_remaining[selected_genres].sum(axis = 1) > 0]
            df_candidates['weights'] = df_candidates[selected_genres[1]] * 3 + df_candidates[selected_genres[0]] * 2
            df_candidates = df_candidates.sort_values('weights', ascending = False)
            df_selected = df_candidates[df_candidates.columns[:-1]].iloc[:N]                        # generated list
            
            df_reacted = df_selected[df_selected['movie_id'].isin(df_interesting['movie_id'])]
            if len(df_reacted) > 0 : 
                f = 1
                df_reacted = df_reacted.sample(n = 1)
            else : f = 0
   
            print("\t\tCount #:",str(count).zfill(len(str(len(df_user)))),"Hit #:",str(f).zfill(len(str(len(df_user)))), end = " ")
            hit_rate = hit_rate + f
            
            df_remaining = df_items[~df_items['movie_id'].isin(df_selected['movie_id'])].reset_index(drop = True)

            rewards = calculate_rewards(rewards, f)
            df_temp = update_grad(predictions,rewards,df_remaining,df_interesting)

            print("Movie #",int(df_temp['movie_id']))
            df_train = analytics.concat([df_train,df_temp])

            X = maths.matrix(df_train[df_train.columns[2:]])[:-1]
            y = maths.matrix(df_train[df_train.columns[2:]])[1:]
            model.fit(X,y, epochs = 20, verbose = False)
            predictions = model.predict(X, verbose = False)
            

        hit_rate = hit_rate / len(df_user)
        hit_rates.append(hit_rate)
        print("Final Rewards :", sum(rewards))
    N_counts.update({N:maths.array(hit_rates)})


For 10

	User # 1 User ID # 36
	Compiling the interesting set ... Compiled
		Count #: 00 Hit #: 00 Movie # 894
		Count #: 01 Hit #: 00 Movie # 539
		Count #: 02 Hit #: 00 Movie # 243
		Count #: 03 Hit #: 00 Movie # 400
		Count #: 04 Hit #: 00 Movie # 400
		Count #: 05 Hit #: 00 Movie # 261
		Count #: 06 Hit #: 00 Movie # 259
		Count #: 07 Hit #: 00 Movie # 795
		Count #: 08 Hit #: 00 Movie # 383
		Count #: 09 Hit #: 00 Movie # 1032
		Count #: 10 Hit #: 00 Movie # 932
		Count #: 11 Hit #: 00 Movie # 63
		Count #: 12 Hit #: 00 Movie # 259
		Count #: 13 Hit #: 00 Movie # 225
		Count #: 14 Hit #: 00 Movie # 932
		Count #: 15 Hit #: 00 Movie # 996
		Count #: 16 Hit #: 00 Movie # 243
		Count #: 17 Hit #: 00 Movie # 400
		Count #: 18 Hit #: 00 Movie # 138
		Count #: 19 Hit #: 00 Movie # 225
Final Rewards : 4.784233454094309

	User # 2 User ID # 34
	Compiling the interesting set ... Compiled
		Count #: 00 Hit #: 00 .Movie # 1017
		Count #: 01 Hit #: 00 Movie # 929
		Count #: 02 Hit #: 00 Movi

In [9]:
maths.array(hit_rates).mean()

0.07250000000000001

# Values

In [10]:
N_counts

{10: array([0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.  , 0.05, 0.05, 0.05,
        0.05, 0.95, 0.05, 0.  , 0.  , 0.05, 0.05, 0.  , 0.05]),
 15: array([0.  , 0.  , 0.05, 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.05, 0.05,
        0.05, 1.  , 0.05, 0.  , 0.05, 0.05, 0.  , 0.  , 0.05]),
 20: array([0.  , 0.05, 0.05, 0.05, 0.  , 0.  , 0.05, 0.  , 0.  , 0.05, 0.  ,
        0.05, 0.95, 0.  , 0.05, 0.05, 0.05, 0.05, 0.  , 0.  ])}

In [11]:
df_performance = analytics.DataFrame(data = N_counts)
df_performance.mean()

10    0.0700
15    0.0750
20    0.0725
dtype: float64